In [70]:
import torch.nn.functional as F
import torch
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import time

In [71]:
class LeNet(torch.nn.Module):

   def __init__(self):
      super(LeNet, self).__init__()
      # Definição de cada camada da rede neural como atributo da classe
      self.conv1 = torch.nn.Conv2d(1, 6, 5)
      self.pool = torch.nn.MaxPool2d(2, 2)
      self.conv2 = torch.nn.Conv2d(6, 16, 5)
      self.fc1 = torch.nn.Linear(16 * 4 * 4, 120)
      self.fc2 = torch.nn.Linear(120, 84)
      self.fc3 = torch.nn.Linear(84, 10)

   def forward(self, x):
      # Computação dos passos de uma rede neural até a saída produzida
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = x.view(-1, 16 * 4 * 4)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

   def num_flat_features(self, x):
      size = x.size()[1:]
      return np.prod(size)

model = LeNet()

In [72]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [73]:
BATCH_SIZE = 4

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

# Create datasets for training & validation, download if necessary
training_set = torchvision.datasets.FashionMNIST(
    './data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST(
    './data', train=False, transform=transform, download=True)

# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = torch.utils.data.DataLoader(
    training_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
validation_loader = torch.utils.data.DataLoader(
    validation_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Class labels
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))


Training set has 60000 instances
Validation set has 10000 instances


In [74]:
info_per_batch = []

In [75]:
def train_one_epoch(epoch_index, tb_writer):
   running_loss = 0.
   last_loss = 0.
   st = time.time()

   # Here, we use enumerate(training_loader) instead of
   # iter(training_loader) so that we can track the batch
   # index and do some intra-epoch reporting
   correct = 0
   for i, data in enumerate(training_loader):
      # Every data instance is an input + label pair
      inputs, labels = data

      # Zero your gradients for every batch!
      optimizer.zero_grad()

      # Make predictions for this batch
      outputs = model(inputs)

      # Compute the loss and its gradients
      loss = loss_fn(outputs, labels)
      loss.backward()

      # Adjust learning weights
      optimizer.step()

      # Gather data and report
      running_loss += loss.item()
      if i % 1000 == 999:
         end = time.time()
         last_loss = running_loss / 1000  # loss per batch
         last_time = (end - st) / 1000  # time per batch
         print('batch {} loss: {}'.format(i + 1, last_loss))
         tb_x = epoch_index * len(training_loader) + i + 1
         tb_writer.add_scalar('Loss/train', last_loss, tb_x)
         running_loss = 0.
         
         _info_per_batch = {}
         _info_per_batch['epoch'] = epoch_index
         _info_per_batch['batch'] = i + 1
         _info_per_batch['mean_loss'] = last_loss
         _info_per_batch['mean_time'] = last_time
         _info_per_batch['time_image_batch'] = last_time / BATCH_SIZE
         info_per_batch.append(_info_per_batch)

         st = time.time()
         

   return last_loss


In [76]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 1

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
   print('EPOCH {}:'.format(epoch_number + 1))

   # Make sure gradient tracking is on, and do a pass over the data
   model.train(True)
   avg_loss = train_one_epoch(epoch_number, writer)

   # We don't need gradients on to do reporting
   model.train(False)

   running_vloss = 0.0
   for i, vdata in enumerate(validation_loader):
      vinputs, vlabels = vdata
      voutputs = model(vinputs)
      vloss = loss_fn(voutputs, vlabels)
      running_vloss += vloss

   avg_vloss = running_vloss / (i + 1)
   print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

   # Log the running loss averaged per batch
   # for both training and validation
   writer.add_scalars('Training vs. Validation Loss',
                     {'Training': avg_loss, 'Validation': avg_vloss},
                     epoch_number + 1)
   writer.flush()

   # Track best performance, and save the model's state
   if avg_vloss < best_vloss:
      best_vloss = avg_vloss
      model_path = 'model_{}_{}'.format(timestamp, epoch_number)
      torch.save(model.state_dict(), model_path)

   epoch_number += 1

print(info_per_batch[0:3])


EPOCH 1:
batch 1000 loss: 2.0269372535049914
batch 2000 loss: 0.8538432704564184
batch 3000 loss: 0.7293968826439232
batch 4000 loss: 0.6106824914133177
batch 5000 loss: 0.5796426721394528
batch 6000 loss: 0.5445708718372043
batch 7000 loss: 0.5065915509434417
batch 8000 loss: 0.467756498349132
batch 9000 loss: 0.4955452290140092
batch 10000 loss: 0.461220390404691
batch 11000 loss: 0.42986467844969595
batch 12000 loss: 0.44380887440266087
batch 13000 loss: 0.4026490962294047
batch 14000 loss: 0.41980449695765854
batch 15000 loss: 0.41401716355385726
LOSS train 0.41401716355385726 valid 0.42975226044654846
[{'epoch': 0, 'batch': 1000, 'mean_loss': 2.0269372535049914, 'mean_time': 0.00681434440612793, 'time_image_batch': 0.0017035861015319825}, {'epoch': 0, 'batch': 2000, 'mean_loss': 0.8538432704564184, 'mean_time': 0.003547818422317505, 'time_image_batch': 0.0008869546055793763}, {'epoch': 0, 'batch': 3000, 'mean_loss': 0.7293968826439232, 'mean_time': 0.00389200496673584, 'time_image